<a href="https://colab.research.google.com/github/hmcarrasco/fcc_text_classifier/blob/main/fcc_Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-07-07 17:29:21--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.08s   

2024-07-07 17:29:21 (4.45 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2024-07-07 17:29:21--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=["Classifications", "Messages"]).dropna()
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=["Classifications", "Messages"]).dropna()

In [ ]:
df_train.head()

,Classifications,Messages
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df_test.head()

,Classifications,Messages
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
df_train['Classifications'] = pd.factorize(df_train['Classifications'])[0]
df_test['Classifications'] = pd.factorize(df_test['Classifications'])[0]
df_test.tail()

,Classifications,Messages
1387,0,true dear..i sat to pray evening and felt so.s...
1388,0,"what will we do in the shower, baby?"
1389,0,where are you ? what are you doing ? are yuou ...
1390,1,ur cash-balance is currently 500 pounds - to m...
1391,1,not heard from u4 a while. call 4 rude chat pr...


In [ ]:
train_labels = df_train['Classifications'].values
test_labels = df_test['Classifications'].values

train_ds = tf.data.Dataset.from_tensor_slices((df_train["Messages"].values, train_labels))

test_ds = tf.data.Dataset.from_tensor_slices((df_test["Messages"].values, test_labels))

In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
VOCAB_SIZE = 1000
MAX_SEQUENCE_LENGTH = 100

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
vectorizer = TextVectorization(
    output_mode='int',
    max_tokens=VOCAB_SIZE,
    output_sequence_length=MAX_SEQUENCE_LENGTH,
)

vectorizer.adapt(train_ds.map(lambda text, label: text))

In [ ]:
model = tf.keras.Sequential([
    vectorizer,
    tf.keras.layers.Embedding(len(vectorizer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=15,
)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


131/131 [==============================] - 34s 133ms/step - loss: 0.6116 - accuracy: 0.8528 - val_loss: 0.4916 - val_accuracy: 0.8604
Epoch 2/15
131/131 [==============================] - 8s 58ms/step - loss: 0.3924 - accuracy: 0.8677 - val_loss: 0.2465 - val_accuracy: 0.8885
Epoch 3/15
131/131 [==============================] - 7s 50ms/step - loss: 0.1578 - accuracy: 0.9624 - val_loss: 0.0933 - val_accuracy: 0.9781
Epoch 4/15
131/131 [==============================] - 6s 44ms/step - loss: 0.0868 - accuracy: 0.9813 - val_loss: 0.0708 - val_accuracy: 0.9844
Epoch 5/15
131/131 [==============================] - 6s 48ms/step - loss: 0.0670 - accuracy: 0.9864 - val_loss: 0.0643 - val_accuracy: 0.9875
Epoch 6/15
131/131 [==============================] - 5s 40ms/step - loss: 0.0538 - accuracy: 0.9892 - val_loss: 0.0604 - val_accuracy: 0.9854
Epoch 7/15
131/131 [==============================] - 5s 40ms/step - loss: 0.0434 - accuracy: 0.9904 - val_loss: 0.0586 - val_accuracy: 0.9854
Epoch 8/

In [ ]:
def predict_message(pred_text):
    pred_prob = model.predict([pred_text])[0][0]
    pred_class = "spam" if pred_prob >= 0.5 else "ham"
    return [pred_prob, pred_class]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460 4",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 [==============================] - 0s 33ms/step
You passed the challenge. Great job!
